In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

# === Настройки устройства ===
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# === Модель ===
model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True
).to(device)

processor = AutoProcessor.from_pretrained(model_id)

# === Создаем пайплайн ===
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

# === Путь к файлу ===
audio_path = "/home/ppwwrr/project_25/voices/voice-2.wav"

# === Транскрибируем ===
print(f"🎧 Обрабатываем файл: {audio_path}")

# `return_timestamps=True` активирует long-form режим
result = pipe(
    audio_path,
    return_timestamps=True,      # ✅ обязательно для длинных аудио
    chunk_length_s=30,           # обрабатывает кусками по 30 секунд
    stride_length_s=(5, 5)       # перекрытие между чанками (для точности)
)

print("\n✅ Транскрипция завершена:\n")
print(result["text"])


Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/2868

🎧 Обрабатываем файл: /home/ppwwrr/project_25/voices/voice-2.wav

✅ Транскрипция завершена:

 Потому что последний раз я смотрела, короче, у меня вот эти духи появились, чтобы не соврать, в 10-м или в 11-м классе, вот типа вот либо конец 10-го, либо начало 11-го, ну короче, вот они у меня появились, мне мама заказала, потому что это духи Полининого отца, вот. Потому что Полина Унисакс Она их носила, мне так понравилось, я такая, блин, что это, я пришла к ней домой, сфоткала, и все, и мы их купили, и еще тогда я смотрела, типа, на золотом яблоке они стоили, типа, ну, 19, ну, 18, ну, как как бы, да, дорого Ну, нишевая парфюмерия Как бы, ладно Ещё можно как бы схавать Но сорок пять, блядь
